In [37]:
import numpy as np

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import numpy as np
import requests
import os

```
accuracy = eval_clf_performance(args)
    cosine_distance = eval_mapper_performance(args)
    accuracy_score = (
        0
        if accuracy < BASE_ACCURACY - ACCURACY_DROP_THRESHOLD
        else (
            1
            if accuracy > BASE_ACCURACY
            else (accuracy - BASE_ACCURACY + ACCURACY_DROP_THRESHOLD)
            / ACCURACY_DROP_THRESHOLD
        )
    )
return accuracy_score + cosine_distance
```

In [9]:
data = np.load(
    "../data/DefenseTransformationEvaluate.npz"
)
print(data["labels"], data["representations"].shape)

data = np.load("../data/DefenseTransformationSubmit.npz")
print(data["representations"].shape)

[7 8 5 ... 0 5 7] (5000, 192)
(20250, 192)


In [18]:
# Load data from the provided file
data = np.load('../data/DefenseTransformationEvaluate.npz')
X = data['representations']
y = data['labels']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classifier on clean representations
classifier_clean = SVC()
classifier_clean.fit(X_train, y_train)

# Evaluate classifier performance on clean representations
y_pred_clean = classifier_clean.predict(X_test)
accuracy_clean = accuracy_score(y_test, y_pred_clean, normalize = True)
print("Accuracy on clean representations:", accuracy_clean)

Accuracy on clean representations: 0.951


In [22]:
def apply_defense_transformation(X):
    # random transformation
    np.random.seed(0)
    X = X + np.random.normal(0, 1, X.shape)
    return X

In [23]:
# Apply defense transformation (e.g., your transformation)
transformed_X_train = apply_defense_transformation(X_train)
transformed_X_test = apply_defense_transformation(X_test)

# Train a classifier on transformed representations
classifier_transformed = SVC()
classifier_transformed.fit(transformed_X_train, y_train)

# Evaluate classifier performance on transformed representations
y_pred_transformed = classifier_transformed.predict(transformed_X_test)
accuracy_transformed = accuracy_score(y_test, y_pred_transformed)
print("Accuracy on transformed representations:", accuracy_transformed)

Accuracy on transformed representations: 0.949


In [27]:
data = np.savez(
    "../data/example_submission.npz",
    representations=apply_defense_transformation(data["representations"]),
)

In [59]:
SERVER_URL = "http://34.71.138.79:9090"
TEAM_TOKEN = "l5pvMfL4ZID1QHmn"

In [60]:
def defense_submit(path_to_npz_file: str):
    endpoint = "/defense/submit"
    url = SERVER_URL + endpoint
    with open(path_to_npz_file, "rb") as f:
        response = requests.post(url, files={"file": f}, headers={"token": TEAM_TOKEN})
        if response.status_code == 200:
            print("Request ok")
            print(response.json())
        else:
            raise Exception(
                f"Defense submit failed. Code: {response.status_code}, content: {response.json()}"
            )

In [61]:
defense_submit("../data/example_submission.npz")

ConnectionError: HTTPConnectionPool(host='34.71.138.79', port=9090): Max retries exceeded with url: /defense/submit (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7d38ac4d5790>: Failed to establish a new connection: [Errno 111] Connection refused'))